# Adding image save button

In [1]:
import threading

from IPython.display import display,Image, clear_output
import ipywidgets


import logging
import numpy as np
import cv2
from collections import deque
import skimage
import time
from datetime import datetime

import os

In [2]:
class OutputWidgetHandler(logging.Handler):
    """
    Custom logging handler sending logs to an output widget
    https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html#Integrating-output-widgets-with-the-logging-module:
    """
    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '80%',
            'height': '100px',
            'border': '1px solid black'
        }
        #self.out = ipywidgets.Output(layout=layout)
        self.out = ipywidgets.Textarea(layout=layout)
        #self.out = ipywidgets.Output()

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record) + "\n"
        #new_output = {
        #    'name': 'stdout',
        #    'output_type': 'stream',
        #    'text': formatted_record+'\n'
        #}
        #self.out.outputs = (new_output, ) + self.out.outputs
        self.out.value = formatted_record + self.out.value
        #memory limit. limit to 1Mb?
        self.out.value[:1000000]

    def show_logs(self):
        """ Show the logs """
        #prob not used?
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()

In [3]:
def gui_simple(image_analysis_func = None):    
    def core(camera):
        def _capture(_):
            '''
            capture the current input image and save to a file 
            '''
            filename = datetime.now().strftime("%Y%m%d_%H%M%S")
            orig_file = os.path.join(path,filename + "_1.jpg")
            cv2.imwrite(os.path.join(path,orig_file),orig_image)
            
            if (image_analysis_func is not None) and image_analysis_widget.value ==True:
                #save the image analysed fle
                processed_file = os.path.join(path,filename + "_2.jpg")                
                cv2.imwrite(os.path.join(path,processed_file),image)
                logger.info("images saved as %s and %s", orig_file,processed_file)
            else:
                logger.info("images saved as %s", orig_file)
                
        def _wrapper(func,image):
            '''
            wrapper of a image analysis function so that image, log will always be outputed
            '''
            ret = func(image)
            if type(ret).__module__ == np.__name__: # if returned value is a numpy (which means, image only)
                log = "" # pseudo log value
                image = ret
            elif type(ret) == tuple: # if returned value is multiple, ex. image, log 
                image, log = ret            
            return image, log
        
        def live(state_widget, camera,image_widget, log_widget,acquire_image_widget):
            nonlocal fps, q, n_frame, orig_image, image
            while state_widget.value == 'connect':
                _, image = camera.read()
                if image is not None:
                    orig_image = image.copy()
                    if (image_analysis_func is not None) and image_analysis_widget.value ==True:
                        image, log = _wrapper(image_analysis_func, image)                        
                    image_widget.value = bytes(cv2.imencode('.jpg', image)[1])
                
                #calculate fps
                now = time.time()
                fps = n_frame / (now - q.popleft())
                q.append(now)
                fps_widget.value = "fps: " + str(fps)
                
                acquire_image_widget.on_click(_capture)
                

        def flag(change):
            nonlocal camera
            
            if change['new'] == 'exit':
                camera.release()
                clear_output()
                #gc.collect()
                print("program ended properly")
                
            if change['new'] == 'disconnect':
                image_widget.value = bytes(cv2.imencode('.jpg',image)[1])
                camera.release()
                logger.info('Camera Disconnected')
                
            if change['new'] == 'connect':
                if camera.isOpened() == False:
                    camera = cv2.VideoCapture(0)
                    
                args = (state_widget, camera, image_widget,handler.out,acquire_image_widget)
                execute_thread = threading.Thread(target=live, args=args)
                execute_thread.start() 
                logger.info('Starting program')

            if change['new'] == 'pause':
                logger.info('Pausing program')
                pass

        #splash = skimage.data.camera()[...,::-1]
        splash = np.zeros(shape=(100,200,3))
        
        #fps calculation
        fps = 0
        n_frame = 3
        q = deque([time.time() for i in range(n_frame)])
        image = splash
        orig_image = splash

        #widgets else than log 
        state_widget = ipywidgets.ToggleButtons(options=['exit','disconnect','pause', 'connect'], description='', value='pause')
        state_widget.observe(flag, names='value')
        
        image_widget = ipywidgets.Image(format='jpeg',value=bytes(cv2.imencode('.jpg',image)[1]), layout={'width':'80%'})
        fps_widget = ipywidgets.Text(value="fps: "+str(fps))
        image_analysis_widget = ipywidgets.Checkbox(value=False,description="apply image analysis")
        
        acquire_image_widget = ipywidgets.Button(description="acquire image", tooltip="acquire image")

        live_execution_widget = ipywidgets.VBox([
            state_widget,
            acquire_image_widget,
            image_analysis_widget,
            image_widget, #main
            fps_widget, #fps
            handler.out, #log
        ])

        display(live_execution_widget)

    
    #initialization
    
    #log function
    logger = logging.getLogger(__name__)
    handler = OutputWidgetHandler() #handler.out is the ipywidgets.Textarea
    handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    #handler.clear_logs()
    
    path = os.path.join(os.getcwd(),"data")
    if not os.path.exists(path):
        os.makedirs(path)
    logger.info("images will be saved to %s", path)
    
    try:
        camera = cv2.VideoCapture(0)
    except:
        #sometimes the camera is already opened
        pass
    assert camera.isOpened(), print("camera not detected")
    core(camera)


In [4]:
def image_filter(image):
    '''
    input image is a BGR image with a dtype of uint8 from cv2.VideoCapture
    image to be returned should also be a BGR of uint8.
    becareful if you use skimage function, as they prefer float.
    the returned value should be
        image (height,width,channel)
        or
        image and string(for logging inthe log window)
    '''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #convert to grayscale image
    return image


In [5]:
gui_simple(image_filter)

program ended properly


# Browse Function

In [6]:
def browse(image_dir):
    def on_change(change):
        #print(change)
        matches = [x for x in files if x.startswith(change["new"])]
        #print(change["new"])
        images = [cv2.imread(os.path.join("./data",x)) for x in matches]
        image = cv2.hconcat([x for x in images])
        image_widget.value = bytes(cv2.imencode('.jpg', image)[1])
        
    files = os.listdir(image_dir)
    unique_files = sorted(set([x[:-6] for x in files]))
    #suffix = _1.jpg so :-6 makes it unique
    
    dropdown_widget = ipywidgets.Dropdown(
        options=unique_files,
        value=unique_files[0],
        description='File Name:',
        disabled=False,)
    dropdown_widget.observe(on_change, names='value')
    
    matches = [x for x in files if x.startswith(dropdown_widget.value)]
    images = np.array([cv2.imread(os.path.join("./data",x)) for x in matches] )
    image = cv2.hconcat([x for x in images])
    
    image_widget = ipywidgets.Image(format='jpeg',value=bytes(cv2.imencode('.jpg',image)[1]), layout={'width':'80%'})
    
    widgets = ipywidgets.VBox([
        dropdown_widget,
        image_widget
        ])
    display(widgets)
    
browse("./data")
    